# Neural network prediction for the sPOD-DL-ROM for WIldfire

In [ ]:
import sys
sys.path.append('../DL/')
sys.path.append('../sPOD/lib/')
sys.path.append('../DL-ROM/LIB/')

In [ ]:
from wildfire1D import wildfire1D
import numpy as np

## Data generation / Shifted POD of the data

In [ ]:
df = wildfire1D(spod_iter=5, var=0)

In [ ]:
impath = "./wildfire_data/save_Wildfire/"
import os
import pickle
os.makedirs(impath, exist_ok=True)

np.save(impath + 'TA_TRAIN.npy', df.TA_TRAIN)
np.save(impath + 'SHIFTS_TRAIN.npy', df.SHIFTS_TRAIN)
np.save(impath + 'PARAMS_TRAIN.npy', df.PARAMS_TRAIN)
np.save(impath + 'U_POD_TRAIN.npy', df.U_POD_TRAIN)
np.save(impath + 'TA_POD_TRAIN.npy', df.TA_POD_TRAIN)
with open(impath + 'U_list.data', 'wb') as filehandle:
    pickle.dump(df.U_list, filehandle)
with open(impath + 'spodModes.data', 'wb') as filehandle:
    pickle.dump(df.spodModes, filehandle)
with open(impath + 'frame_amplitude_list.data', 'wb') as filehandle:
    pickle.dump(df.frame_amplitude_list, filehandle)
with open(impath + 'shifts_list.data', 'wb') as filehandle:
    pickle.dump(df.shifts_list, filehandle)
np.save(impath + 'x.npy', df.x)
np.save(impath + 't.npy', df.t)
np.save(impath + 'MU_VECS_TRAIN.npy', df.mu_vecs_train)

# Load the data for NN predictions and analysis

## Load the training data

In [ ]:
impath = "./wildfire_data/save_Wildfire/"
import os
import pickle

TA_TRAIN = np.load(impath + 'TA_TRAIN.npy')
SHIFTS_TRAIN = np.load(impath + 'SHIFTS_TRAIN.npy')
PARAMS_TRAIN = np.load(impath + 'PARAMS_TRAIN.npy')
U_POD_TRAIN = np.load(impath + 'U_POD_TRAIN.npy')
TA_POD_TRAIN = np.load(impath + 'TA_POD_TRAIN.npy')
with open(impath + 'U_list.data', 'rb') as filehandle:
    U_list = pickle.load(filehandle) 
with open(impath + 'spodModes.data', 'rb') as filehandle:
    spodModes = pickle.load(filehandle) 
with open(impath + 'frame_amplitude_list.data', 'rb') as filehandle:
    frame_amplitude_list = pickle.load(filehandle) 
with open(impath + 'shifts_list.data', 'rb') as filehandle:
    shifts_list = pickle.load(filehandle) 
x = np.load(impath + 'x.npy')
t = np.load(impath + 't.npy')
MU_VECS_TRAIN = np.load(impath + 'MU_VECS_TRAIN.npy')

In [ ]:
# Temporary cell for dof study.

frame_wise_sPOD = [15, 20, 15]
Nmf = spodModes
time_amplitudes_1 = TA_TRAIN[:Nmf[0], :]
time_amplitudes_2 = TA_TRAIN[Nmf[0]:Nmf[0] + Nmf[1], :]
time_amplitudes_3 = TA_TRAIN[Nmf[0] + Nmf[1]:, :]

frame_amplitudes_training = [
    time_amplitudes_1[:frame_wise_sPOD[0], :],
    time_amplitudes_2[:frame_wise_sPOD[1], :],
    time_amplitudes_3[:frame_wise_sPOD[2], :]
]
TA_TRAIN = np.concatenate(frame_amplitudes_training, axis=0)
U_list = [
    U_list[0][:, :frame_wise_sPOD[0]], 
    U_list[1][:, :frame_wise_sPOD[1]], 
    U_list[2][:, :frame_wise_sPOD[2]]
]
spodModes = frame_wise_sPOD

frame_amplitude_list = []
Nsamples_train = 3
for frame in range(3):
    Nmodes = spodModes[frame]
    VT = frame_amplitudes_training[frame]
    amplitudes = [np.reshape(VT[n, :], [Nsamples_train, len(t)]).T for n in range(Nmodes)]
    frame_amplitude_list.append(amplitudes)

    
    
frame_wise_POD = sum(frame_wise_sPOD) + 2
U_POD_TRAIN = U_POD_TRAIN[:, :frame_wise_POD]
TA_POD_TRAIN = TA_POD_TRAIN[:frame_wise_POD, :]

In [ ]:

import matplotlib.pyplot as plt
import os
from sklearn.utils.extmath import randomized_svd
from Helper import *
############################################
data_shape = [len(x), 1, 1, 3*len(t)]
dx = x[1] - x[0]
L = [x[-1]]

q_train = [U_list[0] @ frame_amplitudes_training[0], 
          U_list[1] @ frame_amplitudes_training[1], 
          U_list[2] @ frame_amplitudes_training[2]]
trafos = [
transforms(data_shape, L + dx, shifts=SHIFTS_TRAIN[0], dx=[dx],
                          use_scipy_transform=False,
                          interp_order=5),
transforms(data_shape, L + dx, shifts=np.zeros_like(SHIFTS_TRAIN[0]), trafo_type="identity",
                          dx=[dx],
                          use_scipy_transform=False, interp_order=5),
transforms(data_shape, L + dx, shifts=SHIFTS_TRAIN[1], dx=[dx],
                          use_scipy_transform=False,
                          interp_order=5)
]

NumFrames = 3
q_sPOD = 0
for frame in range(NumFrames):
    q_sPOD += trafos[frame].apply(q_train[frame])

############################################
q_POD = U_POD_TRAIN @ TA_POD_TRAIN
############################################
import os
dat1_train = np.load(os.path.abspath(".") + '/wildfire_data/' + 'SnapShotMatrix540.npy')
dat2_train = np.load(os.path.abspath(".") + '/wildfire_data/' + 'SnapShotMatrix560.npy')
dat3_train = np.load(os.path.abspath(".") + '/wildfire_data/' + 'SnapShotMatrix580.npy')
q_train_total = np.concatenate((dat1_train, dat2_train, dat3_train), axis=1)
Q_original = q_train_total[0:len(x), :]
############################################

num1 = np.sqrt(np.mean(np.linalg.norm(Q_original - q_sPOD, 2, axis=1) ** 2))
den1 = np.sqrt(np.mean(np.linalg.norm(Q_original, 2, axis=1) ** 2))

num2 = np.sqrt(np.mean(np.linalg.norm(Q_original - q_POD, 2, axis=1) ** 2))
den2 = np.sqrt(np.mean(np.linalg.norm(Q_original, 2, axis=1) ** 2))

print(num1/den1)
print(num2/den2)

## Load the testing data

In [ ]:
import os
from wildfire1D import wildfire1D_testing

Q = np.load(os.path.abspath(".") + '/wildfire_data/' + 'SnapShotMatrix558_49.npy')
delta = np.load(os.path.abspath(".") + '/wildfire_data/' + 'Shifts558_49.npy')


###########################################
Q = Q[:, :len(t)]
delta = delta[:, :len(t)]
###########################################

TA_TEST, TA_POD_TEST, SHIFTS_TEST, PARAMS_TEST, MU_VECS_TEST, q_test, q1_test, q2_test, q3_test = wildfire1D_testing(Q, delta, U_list, U_POD_TRAIN, x, t, val=558.49, var=0, spod_iter=5)

In [ ]:
impath = "./wildfire_data/save_Wildfire/"
import os
import pickle
os.makedirs(impath, exist_ok=True)

np.save(impath + 'TA_TEST.npy', TA_TEST)
np.save(impath + 'TA_POD_TEST.npy', TA_POD_TEST)
np.save(impath + 'SHIFTS_TEST.npy', SHIFTS_TEST)
np.save(impath + 'PARAMS_TEST.npy', PARAMS_TEST)
np.save(impath + 'MU_VECS_TEST.npy', MU_VECS_TEST)
np.save(impath + 'q_test.npy', q_test)
np.save(impath + 'q1_test.npy', q1_test)
np.save(impath + 'q2_test.npy', q2_test)
np.save(impath + 'q3_test.npy', q3_test)

In [ ]:
impath = "./wildfire_data/save_Wildfire/"
import os
import pickle

TA_TEST = np.load(impath + 'TA_TEST.npy')
TA_POD_TEST = np.load(impath + 'TA_POD_TEST.npy')
SHIFTS_TEST = np.load(impath + 'SHIFTS_TEST.npy')
PARAMS_TEST = np.load(impath + 'PARAMS_TEST.npy')
MU_VECS_TEST = np.load(impath + 'MU_VECS_TEST.npy')
q_test = np.load(impath + 'q_test.npy')
q1_test = np.load(impath + 'q1_test.npy')
q2_test = np.load(impath + 'q2_test.npy')
q3_test = np.load(impath + 'q3_test.npy')

In [ ]:
# Temporary cell for time study
time_amplitudes_1_test = U_list[0].transpose() @ q1_test
time_amplitudes_2_test = U_list[1].transpose() @ q2_test
time_amplitudes_3_test = U_list[2].transpose() @ q3_test
amplitudes_test = np.concatenate(
    (time_amplitudes_1_test, time_amplitudes_2_test, time_amplitudes_3_test), axis=0)

TA_TEST = amplitudes_test

TA_POD_TEST = U_POD_TRAIN.transpose() @ q_test

# -----------------------------------------------------------

In [ ]:
import matplotlib.pyplot as plt
import os
from sklearn.utils.extmath import randomized_svd
from Helper import *

U, S, VT = randomized_svd(q_test, n_components=6)
q_POD = U @ (np.diag(S) @ VT)


############################################
data_shape = [len(x), 1, 1, len(t)]
dx = x[1] - x[0]
L = [x[-1]]

q_pred = [q1_test, q2_test, q3_test]
trafos = [
transforms(data_shape, L + dx, shifts=SHIFTS_TEST[0], dx=[dx],
                          use_scipy_transform=False,
                          interp_order=5),
transforms(data_shape, L + dx, shifts=np.zeros_like(SHIFTS_TEST[0]), trafo_type="identity",
                          dx=[dx],
                          use_scipy_transform=False, interp_order=5),
transforms(data_shape, L + dx, shifts=SHIFTS_TEST[1], dx=[dx],
                          use_scipy_transform=False,
                          interp_order=5)
]

NumFrames = 3
q_sPOD = 0
for frame in range(NumFrames):
    q_sPOD += trafos[frame].apply(q_pred[frame])

############################################


[X_grid, t_grid] = np.meshgrid(x, t)
X_grid = X_grid.T
t_grid = t_grid.T
impath = "../plots/images_wildfire1D/"
os.makedirs(impath, exist_ok=True)
fig, axs = plt.subplots(1, 3, sharey=True)
vmin = np.min(q_test)
vmax = np.max(q_test)
# Original
axs[0].pcolormesh(X_grid, t_grid, q_test, vmin=vmin, vmax=vmax)
axs[0].set_ylabel(r"$t$")
axs[0].set_yticks([], [])
axs[0].set_xlabel(r'$x$')
axs[0].set_xticks([], [])
axs[0].set_title(r"$Q$")
# POD
axs[1].pcolormesh(X_grid, t_grid, q_POD, vmin=vmin, vmax=vmax)
axs[1].set_yticks([], [])
axs[1].set_xlabel(r'$x$')
axs[1].set_xticks([], [])
axs[1].set_title(r"$Q^" + "{POD}$")
# sPOD
axs[2].pcolormesh(X_grid, t_grid, q_sPOD, vmin=vmin, vmax=vmax)
axs[2].set_xlabel(r'$x$')
axs[2].set_yticks([], [])
axs[2].set_xticks([], [])
axs[2].set_title(r"$Q^" + "{sPOD}$")
plt.tight_layout()

save_fig(filepath=impath + "PODvsPOD", figure=fig)

In [ ]:
import matplotlib.pyplot as plt
from Helper import *
impath = "../plots/images_wildfire1D/"

truncated_modes = np.array([0, 1, 3, 6, 15, 20, 35, 50, 64])
recon_err_POD = np.array([1, 0.86462, 0.55, 0.38, 0.22, 0.16, 0.07, 0.038, 0.021])
recon_err_sPOD = np.array([1, 0.86462, 0.41, 0.41, 0.025, 0.024, 0.0015, 0.00025, 0.0000528])
fig = plt.figure()
plt.semilogy(truncated_modes, recon_err_POD, color="red", marker=".", label='POD')
plt.semilogy(truncated_modes, recon_err_sPOD, color="blue", marker=".", label='sPOD')
plt.xlabel('Number of modes', fontsize=14)
plt.ylabel('Relative reconstruction error', fontsize=14)
plt.title('Modes vs error', fontsize=14)
plt.grid()
plt.legend(loc='center right')
save_fig(filepath=impath + 'MvE', figure=fig)

In [ ]:
impath = "./wildfire_data/save_Wildfire/"
import os
import matplotlib.pyplot as plt
from Helper import *

Q = np.load(os.path.abspath(".") + '/wildfire_data/' + 'SnapShotMatrix558_49.npy')

[X_grid, t_grid] = np.meshgrid(x, t)
X_grid = X_grid.T
t_grid = t_grid.T
impath = "../plots/images_wildfire1D/"
os.makedirs(impath, exist_ok=True)
fig, axs = plt.subplots(1, 2, sharey=True)
# Original
axs[0].pcolormesh(X_grid, t_grid, Q[:len(x), :])
# axs[0].set_ylabel(r"$t$")
axs[0].set_yticks([], [])
# axs[0].set_xlabel(r'$x$')
axs[0].set_xticks([], [])
axs[0].set_title(r"$T$")
# POD
axs[1].pcolormesh(X_grid, t_grid, Q[len(x):, :])
axs[1].set_yticks([], [])
# axs[1].set_xlabel(r'$x$')
axs[1].set_xticks([], [])
axs[1].set_title(r"$S$")

plt.tight_layout()

save_fig(filepath=impath + "TvS", figure=fig)

# -----------------------------------------------------------

In [ ]:
print("Grid, Nx : {}, Nt : {}".format(len(x), len(t)))
print("Number of sPOD frames : {}".format(len(spodModes)))
print("Number of modes (frame wise) : {}, {}, {}".format(spodModes[0], spodModes[1], spodModes[2]))
print("Size of training matrix : {} x {}".format(int(TA_TRAIN.shape[0]), int(TA_TRAIN.shape[1])))

In [ ]:
# This cell is reserved for any data manipulations that need to be done for the network input

# "0" shift has not been included in the prediction 
shifts_train = np.concatenate((np.reshape(SHIFTS_TRAIN[0], newshape=[1, -1]), np.reshape(SHIFTS_TRAIN[1], newshape=[1, -1])), axis=0)
shifts_test = np.concatenate((np.reshape(SHIFTS_TEST[0], newshape=[1, -1]), np.reshape(SHIFTS_TEST[1], newshape=[1, -1])), axis=0)

ta_train = np.concatenate((TA_TRAIN, shifts_train), axis=0)
ta_test = np.concatenate((TA_TEST, shifts_test), axis=0)

# Network call

# Feed forward neural network

In [ ]:
# Path for the pretrained weights
PATH_sPOD = 'DNN_result/wildfire1D/training_results_sPOD/T/2022_08_29__11-29-59/trained_weights/weights.pt'
PATH_POD = 'DNN_result/wildfire1D/training_results_POD/T/2022_08_29__12-14-13/trained_weights/weights.pt'

In [ ]:
params_sPOD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': len(x),  # N_h
        'reduced_order_model_dimension': ta_train.shape[0],  # N
        'totalModes': ta_train.shape[0] - len(spodModes) - 1,  # Total number of modes for all the frames
    }
params_POD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': len(x),  # N_h
        'reduced_order_model_dimension': TA_POD_TRAIN.shape[0],  # N
        'totalModes': TA_POD_TRAIN.shape[0],  # Total number of modes for all the frames
    }

In [ ]:
# training the model
from network import run_model 
print("#################################")
print("sPOD-DL-ROM")
model_sPOD, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, epochs=100, lr=0.05, loss='L1', 
                       logs_folder='./DNN_result/wildfire1D/training_results_sPOD/T',
                      pretrained_load=False, pretrained_weights=PATH_sPOD, params=params_sPOD)
print("#################################\n")
print("#################################")
print("POD-DL-ROM")
model_POD, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, epochs=100, lr=0.05, loss='L1', 
                      logs_folder='./DNN_result/wildfire1D/training_results_POD/T',
                     pretrained_load=False, pretrained_weights=PATH_POD, params=params_POD)
print("#################################\n")

In [ ]:
# loading the model
import torch
import pathlib
import os

log_folder_base_sPOD = 'DNN_result/wildfire1D/training_results_sPOD/T/'
log_folder_trained_model_sPOD = sorted(pathlib.Path(log_folder_base_sPOD).glob('*/'), key=os.path.getmtime)[-1]
PATH_sPOD = str(log_folder_trained_model_sPOD) + '/trained_weights/' + 'weights.pt'


log_folder_base_POD = 'DNN_result/wildfire1D/training_results_POD/T/'
log_folder_trained_model_POD = sorted(pathlib.Path(log_folder_base_POD).glob('*/'), key=os.path.getmtime)[-1]
PATH_POD = str(log_folder_trained_model_POD) + '/trained_weights/' + 'weights.pt'


# PATH_sPOD = 'DNN/Wildfire/training_results_local_sPOD/T/2022_09_01__22-55-10/trained_weights/weights.pt'
# PATH_POD = 'DNN/Wildfire/training_results_local_POD/T/2022_09_02__00-03-56/trained_weights/weights.pt'

print(PATH_sPOD)
print(PATH_POD)

In [ ]:
from network import scale_params

if '/trained_weights/weights.pt' in PATH_sPOD: address_sPOD = PATH_sPOD.replace('/trained_weights/weights.pt', '')
scaling_sPOD = np.load(address_sPOD + '/variables/' + 'scaling.npy', allow_pickle=True)

if '/trained_weights/weights.pt' in PATH_POD: address_POD = PATH_POD.replace('/trained_weights/weights.pt', '')
scaling_POD = np.load(address_POD + '/variables/' + 'scaling.npy', allow_pickle=True)

PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, params_sPOD, scaling_sPOD)
PARAMS_TEST_POD = scale_params(PARAMS_TEST, params_POD, scaling_POD)

In [ ]:
print(ta_test.shape)

In [ ]:
# testing the model
from network import test_model 
import time 

tic = time.process_time()
rel_err_sPOD, results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, 
                                                  trained_model=None, saved_model=True,
                                                 PATH_TO_WEIGHTS=PATH_sPOD, params=params_sPOD, 
                                                  scaling=scaling_sPOD) 
toc = time.process_time()
print(f"Time consumption in testing sPOD DL model : {toc - tic:0.4f} seconds")

tic = time.process_time()
rel_err_POD, results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, 
                                                trained_model=None, saved_model=True,
                                               PATH_TO_WEIGHTS=PATH_POD, params=params_POD,
                                               scaling=scaling_POD)
toc = time.process_time()
print(f"Time consumption in testing POD DL model : {toc - tic:0.4f} seconds")


print(rel_err_sPOD, rel_err_POD)

# Gradient boosted regression tree

In [ ]:
params_sPOD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': len(x),  # N_h
        'reduced_order_model_dimension': ta_train.shape[0],  # N
        'totalModes': ta_train.shape[0] - len(spodModes) - 1,  # Total number of modes for all the frames
    }
params_POD = {
        'scaling': True,  # true if the data should be scaled
        'full_order_model_dimension': len(x),  # N_h
        'reduced_order_model_dimension': TA_POD_TRAIN.shape[0],  # N
        'totalModes': TA_POD_TRAIN.shape[0],  # Total number of modes for all the frames
    }

In [ ]:
# training the model
from GBRT import run_model 
print("#################################")
print("sPOD-DL-ROM")
model_sPOD_list, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, params_sPOD)
print("#################################\n")
print("#################################")
print("POD-DL-ROM")
model_POD_list, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, params_POD)
print("#################################\n")

In [ ]:
import pickle

path = "./GBRT/Wildfire/64new/"
os.makedirs(path, exist_ok=True)

with open(path + 'sPOD.pkl', 'wb') as f:
    pickle.dump(model_sPOD_list, f)
    
with open(path + 'POD.pkl', 'wb') as f:
    pickle.dump(model_POD_list, f)
    
with open(path + 'scaling_sPOD.pkl', 'wb') as f:
    pickle.dump(scaling_sPOD, f)
    
with open(path + 'scaling_POD.pkl', 'wb') as f:
    pickle.dump(scaling_POD, f)

In [ ]:
import joblib

path = "./GBRT/Wildfire/64new/"

model_sPOD_list = joblib.load(path + 'sPOD.pkl')
model_POD_list = joblib.load(path + 'POD.pkl')
scaling_sPOD = joblib.load(path + 'scaling_sPOD.pkl')
scaling_POD = joblib.load(path + 'scaling_POD.pkl')

In [ ]:
from GBRT import scale_params
PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, scaling_sPOD)
PARAMS_TEST_POD = scale_params(PARAMS_TEST, scaling_POD)

In [ ]:
# testing the model
from GBRT import test_model 
import time 

print("#################################")
tic = time.process_time()
results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, 
                                                  trained_model_list=model_sPOD_list,
                                                  scaling=scaling_sPOD, params=params_sPOD) 
toc = time.process_time()
print(f"Time consumption in testing sPOD DL model : {toc - tic:0.4f} seconds")
print("#################################\n")
print("#################################")
tic = time.process_time()
results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, 
                                                trained_model_list=model_POD_list,
                                                scaling=scaling_POD, params=params_POD)
toc = time.process_time()
print(f"Time consumption in testing POD DL model : {toc - tic:0.4f} seconds")
print("#################################\n")

# Convolutional autoencoder and DNN coupled model

In [ ]:
from TrainingFramework import TrainingFramework
from TestingFramework import TestingFramework
import Helper

In [ ]:
dict_network_sPOD = {
        'time_amplitude_train': ta_train,
        'time_amplitude_test': ta_test,
        'parameter_train': PARAMS_TRAIN,
        'parameter_test': PARAMS_TEST,
        'batch_size': 500,
        'num_early_stop': 2000,  # Number of epochs for the early stopping
        'pretrained_load': False,  # Wthere to initialize the network with pretrained weights
        'scaling': True,  # true if the data should be scaled
        'perform_svd': 'randomized',  # 'normal', 'randomized'
        'learning_rate': 0.1,  # eta
        'full_order_model_dimension': len(x),  # N_h
        'reduced_order_model_dimension': ta_train.shape[0],  # N
        'encoded_dimension': 4,  # dimension of the system after the encoder
        'omega_h': 0.5,
        'omega_N': 0.5,
        'typeConv': '1D',  # Type of convolutional layer for the network : '1D' or '2D'
        'totalModes': ta_train.shape[0] - len(spodModes) - 1,  # Total number of modes for all the frames
    }


dict_network_POD = {
        'time_amplitude_train': TA_POD_TRAIN,
        'time_amplitude_test': TA_POD_TEST,
        'parameter_train': PARAMS_TRAIN,
        'parameter_test': PARAMS_TEST,
        'batch_size': 500,
        'num_early_stop': 2000,  # Number of epochs for the early stopping
        'pretrained_load': False,  # Wthere to initialize the network with pretrained weights
        'scaling': True,  # true if the data should be scaled
        'perform_svd': 'randomized',  # 'normal', 'randomized'
        'learning_rate': 0.1,  # eta
        'full_order_model_dimension': len(x),  # N_h
        'reduced_order_model_dimension': TA_POD_TRAIN.shape[0],  # N
        'encoded_dimension': 4,  # dimension of the system after the encoder
        'omega_h': 0.5,
        'omega_N': 0.5,
        'typeConv': '1D',  # Type of convolutional layer for the network : '1D' or '2D'
        'totalModes': TA_POD_TRAIN.shape[0],  # Total number of modes for all the frames
    }



## -------------------------------------

In [ ]:
# select the path to the pre trained weights
PATH_sPOD = 'CADNN_result/synthetic/training_results_sPOD/2022_08_23__19-55-17/net_weights/epoch_99.pt'
PATH_POD = 'CADNN_result/synthetic/training_results_POD/2022_08_23__19-55-17/net_weights/epoch_99.pt'

In [ ]:
# Training model for sPOD
train_model_sPOD = TrainingFramework(dict_network_sPOD, split=0.70, 
                                     log_folder='./CADNN_result/wildfire1D/training_results_sPOD/T/')
trained_model_sPOD = train_model_sPOD.training(epochs=100, save_every=50, print_every=50, 
                                               log_base_name='/', pretrained_weights=PATH_sPOD)

In [ ]:
import os
import pathlib

# Testing model for sPOD
testing_method = ''

log_folder_base = 'CADNN_result/wildfire1D/training_results_sPOD/T/'
log_folder_trained_model = sorted(pathlib.Path(log_folder_base).glob('*/'), key=os.path.getmtime)[-1]

test_model_sPOD = TestingFramework(dict_network_sPOD)
test_model_sPOD.testing(log_folder_trained_model=str(log_folder_trained_model), 
                        testing_method=testing_method, model=trained_model_sPOD)
results_predicted_sPOD = test_model_sPOD.time_amplitude_test_output

## -------------------------------------

In [ ]:
# Training model for POD
train_model_POD = TrainingFramework(dict_network_POD, split=0.70, 
                                    log_folder='./CADNN_result/wildfire1D/training_results_POD/T/')
trained_model_POD = train_model_POD.training(epochs=100, save_every=50, print_every=50, 
                                             log_base_name='/', pretrained_weights=PATH_POD)

In [ ]:
import os
import pathlib

# Testing model for POD
testing_method = ''

log_folder_base = 'CADNN_result/wildfire1D/training_results_POD/T/'
log_folder_trained_model = sorted(pathlib.Path(log_folder_base).glob('*/'), key=os.path.getmtime)[-1]

test_model_POD = TestingFramework(dict_network_POD)
test_model_POD.testing(log_folder_trained_model=str(log_folder_trained_model), 
                       testing_method=testing_method, model=trained_model_POD)
results_predicted_POD = test_model_POD.time_amplitude_test_output

## -------------------------------------

# Online error analysis

In [ ]:
# This cell is reserved for data manipulations for the online analysis
frame_amplitudes_predicted_sPOD = results_predicted_sPOD[:-2, :]
shifts_predicted_sPOD = results_predicted_sPOD[-2:, :]
frame_amplitudes_predicted_POD = results_predicted_POD

In [ ]:
from wildfire1D import onlineErroranalysis
tmp = onlineErroranalysis(frame_amplitudes_predicted_sPOD, 
                    TA_TEST,
                    TA_POD_TEST,
                    frame_amplitude_list,
                    shifts_predicted_sPOD,
                    SHIFTS_TEST,
                    shifts_list,
                    frame_amplitudes_predicted_POD,
                    spodModes,
                    U_list,
                    U_POD_TRAIN,
                    q_test,
                    q1_test,
                    q2_test,
                    q3_test,
                    MU_VECS_TRAIN,
                    MU_VECS_TEST,
                    x, t)

In [ ]:
import matplotlib.pyplot as plt
import os
from sklearn.utils.extmath import randomized_svd
from Helper import *
impath = "../plots/images_wildfire1D/"

In [ ]:
truncated_modes = np.array([3, 6, 15, 20, 35, 50, 64])
recon_err_POD = np.array([0.55695, 0.38588, 0.221744, 0.167523, 0.09577, 0.10325, 0.08427])
recon_err_sPOD = np.array([0.99486, 0.96774, 0.18548, 0.18436, 0.06719, 0.0554, 0.03354])
recon_err_interp = np.array([0.27787, 0.2768, 0.05805, 0.05771, 0.05576, 0.05573, 0.05573])
fig = plt.figure()
plt.semilogy(truncated_modes, recon_err_POD, color="green", linestyle='-', marker=".", label='POD')
plt.semilogy(truncated_modes, recon_err_sPOD, color="red", linestyle='--', marker="+", label='sPOD')
plt.semilogy(truncated_modes, recon_err_interp, color="blue", linestyle='--', marker="*", label='Interpolated')
plt.xlabel('Number of modes', fontsize=14)
plt.ylabel('rel. error', fontsize=14)
plt.title('Modes vs error', fontsize=14)
plt.grid()
plt.legend(loc='upper right')
save_fig(filepath=impath + 'MvE_online', figure=fig)

In [ ]:
truncated_modes = np.array([545, 550, 555, 558.49, 565, 570, 575])
recon_err_POD = np.array([0.55695, 0.38588, 0.221744, 0.167523, 0.09577, 0.10325, 0.08427])
recon_err_sPOD = np.array([0.99486, 0.96774, 0.18548, 0.18436, 0.06719, 0.0554, 0.03354])
recon_err_interp = np.array([0.27787, 0.2768, 0.05805, 0.05771, 0.05576, 0.05573, 0.05573])
fig = plt.figure()
plt.semilogy(truncated_modes, recon_err_POD, color="green", linestyle='-', marker=".", label='POD')
plt.semilogy(truncated_modes, recon_err_sPOD, color="red", linestyle='--', marker="+", label='sPOD')
plt.semilogy(truncated_modes, recon_err_interp, color="blue", linestyle='--', marker="*", label='Interpolated')
plt.xlabel(r'$\beta$', fontsize=14)
plt.ylabel('rel. error', fontsize=14)
plt.grid()
plt.legend(loc='upper right')
save_fig(filepath=impath + 'betavE', figure=fig)